In [6]:
import numpy as np
!pip install xgboost
from scipy.stats import uniform, randint
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split

import xgboost as xgb
import pandas as pd
import numpy as np


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for xgboost from https://files.pythonhosted.org/packages/24/ec/ad387100fa3cc2b9b81af0829b5ecfe75ec5bb19dd7c19d4fea06fb81802/xgboost-2.0.3-py3-none-win_amd64.whl.metadata
   ---------------------------------------- 99.8/99.8 MB 1.6 MB/s eta 0:00:00


In [7]:
df = pd.read_csv("final_training.csv")
df

,Residue_Index,Amino_Acid,PDBID,Secondary_Structure,Junction_Region,Arginine_Propensity,Tryptophan_Propensity,Tyrosine_Propensity,Valine_Propensity,Serine_Propensity,Methionine_Propensity,Threonine_Propensity,Leucine_Propensity,Chain,RelSASA,Secondary_Enum,Label
0,1,A,DZI.A,C,C,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,A,0.738996,0.43330,NaN
1,2,L,DZI.A,C,C,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,A,0.417377,0.43330,NaN
2,3,I,DZI.A,C,J,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,A,0.110747,6.27530,NaN
3,4,D,DZI.A,E,J,0.0,0.0,0.0,0.428571,0.000000,0.0,0.000000,0.142857,A,0.094342,6.27530,NaN
4,5,V,DZI.A,E,E,0.0,0.0,0.0,0.571429,0.000000,0.0,0.000000,0.142857,A,0.016116,0.17134,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17990,84,G,E6P.B,C,C,0.0,0.0,0.0,0.000000,0.285714,0.0,0.142857,0.285714,B,0.424894,0.43330,NaN
17991,85,S,E6P.B,C,C,0.0,0.0,0.0,0.000000,0.428571,0.0,0.000000,0.285714,B,0.603341,0.43330,1.0
17992,86,S,E6P.B,C,C,0.0,0.0,0.0,0.000000,0.500000,0.0,0.000000,0.166667,B,0.689852,0.43330,1.0
17993,87,P,E6P.B,C,C,0.0,0.0,0.0,0.000000,0.600000,0.0,0.000000,0.000000,B,0.742278,0.43330,1.0


In [8]:
df['Label'] = df['Label'].replace(np.nan, 0)
df

,Residue_Index,Amino_Acid,PDBID,Secondary_Structure,Junction_Region,Arginine_Propensity,Tryptophan_Propensity,Tyrosine_Propensity,Valine_Propensity,Serine_Propensity,Methionine_Propensity,Threonine_Propensity,Leucine_Propensity,Chain,RelSASA,Secondary_Enum,Label
0,1,A,DZI.A,C,C,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,A,0.738996,0.43330,0.0
1,2,L,DZI.A,C,C,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,A,0.417377,0.43330,0.0
2,3,I,DZI.A,C,J,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,A,0.110747,6.27530,0.0
3,4,D,DZI.A,E,J,0.0,0.0,0.0,0.428571,0.000000,0.0,0.000000,0.142857,A,0.094342,6.27530,0.0
4,5,V,DZI.A,E,E,0.0,0.0,0.0,0.571429,0.000000,0.0,0.000000,0.142857,A,0.016116,0.17134,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17990,84,G,E6P.B,C,C,0.0,0.0,0.0,0.000000,0.285714,0.0,0.142857,0.285714,B,0.424894,0.43330,0.0
17991,85,S,E6P.B,C,C,0.0,0.0,0.0,0.000000,0.428571,0.0,0.000000,0.285714,B,0.603341,0.43330,1.0
17992,86,S,E6P.B,C,C,0.0,0.0,0.0,0.000000,0.500000,0.0,0.000000,0.166667,B,0.689852,0.43330,1.0
17993,87,P,E6P.B,C,C,0.0,0.0,0.0,0.000000,0.600000,0.0,0.000000,0.000000,B,0.742278,0.43330,1.0


In [9]:
aa_to_number = {
    'A': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5,
    'G': 6, 'H': 7, 'I': 8, 'K': 9, 'L': 10,
    'M': 11, 'N': 12, 'P': 13, 'Q': 14, 'R': 15,
    'S': 16, 'T': 17, 'V': 18, 'W': 19, 'Y': 20
}

# Convert Amino_Acid column to numbers
df['Amino_Acid'] = df['Amino_Acid'].map(aa_to_number)
df

,Residue_Index,Amino_Acid,PDBID,Secondary_Structure,Junction_Region,Arginine_Propensity,Tryptophan_Propensity,Tyrosine_Propensity,Valine_Propensity,Serine_Propensity,Methionine_Propensity,Threonine_Propensity,Leucine_Propensity,Chain,RelSASA,Secondary_Enum,Label
0,1,1,DZI.A,C,C,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,A,0.738996,0.43330,0.0
1,2,10,DZI.A,C,C,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,A,0.417377,0.43330,0.0
2,3,8,DZI.A,C,J,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,A,0.110747,6.27530,0.0
3,4,3,DZI.A,E,J,0.0,0.0,0.0,0.428571,0.000000,0.0,0.000000,0.142857,A,0.094342,6.27530,0.0
4,5,18,DZI.A,E,E,0.0,0.0,0.0,0.571429,0.000000,0.0,0.000000,0.142857,A,0.016116,0.17134,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17990,84,6,E6P.B,C,C,0.0,0.0,0.0,0.000000,0.285714,0.0,0.142857,0.285714,B,0.424894,0.43330,0.0
17991,85,16,E6P.B,C,C,0.0,0.0,0.0,0.000000,0.428571,0.0,0.000000,0.285714,B,0.603341,0.43330,1.0
17992,86,16,E6P.B,C,C,0.0,0.0,0.0,0.000000,0.500000,0.0,0.000000,0.166667,B,0.689852,0.43330,1.0
17993,87,13,E6P.B,C,C,0.0,0.0,0.0,0.000000,0.600000,0.0,0.000000,0.000000,B,0.742278,0.43330,1.0


In [10]:
df["Valine_Propensity"]=df["Valine_Propensity"].apply(lambda x:x*-1)
df["Serine_Propensity"]=df["Serine_Propensity"].apply(lambda x:x*-1)
df["Methionine_Propensity"]=df["Methionine_Propensity"].apply(lambda x:x*-1)
df["Threonine_Propensity"]=df["Threonine_Propensity"].apply(lambda x:x*-1)
df["Leucine_Propensity"]=df["Leucine_Propensity"].apply(lambda x:x*-1)
df

,Residue_Index,Amino_Acid,PDBID,Secondary_Structure,Junction_Region,Arginine_Propensity,Tryptophan_Propensity,Tyrosine_Propensity,Valine_Propensity,Serine_Propensity,Methionine_Propensity,Threonine_Propensity,Leucine_Propensity,Chain,RelSASA,Secondary_Enum,Label
0,1,1,DZI.A,C,C,0.0,0.0,0.0,-0.000000,-0.000000,-0.0,-0.000000,-0.000000,A,0.738996,0.43330,0.0
1,2,10,DZI.A,C,C,0.0,0.0,0.0,-0.000000,-0.000000,-0.0,-0.000000,-0.000000,A,0.417377,0.43330,0.0
2,3,8,DZI.A,C,J,0.0,0.0,0.0,-0.000000,-0.000000,-0.0,-0.000000,-0.000000,A,0.110747,6.27530,0.0
3,4,3,DZI.A,E,J,0.0,0.0,0.0,-0.428571,-0.000000,-0.0,-0.000000,-0.142857,A,0.094342,6.27530,0.0
4,5,18,DZI.A,E,E,0.0,0.0,0.0,-0.571429,-0.000000,-0.0,-0.000000,-0.142857,A,0.016116,0.17134,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17990,84,6,E6P.B,C,C,0.0,0.0,0.0,-0.000000,-0.285714,-0.0,-0.142857,-0.285714,B,0.424894,0.43330,0.0
17991,85,16,E6P.B,C,C,0.0,0.0,0.0,-0.000000,-0.428571,-0.0,-0.000000,-0.285714,B,0.603341,0.43330,1.0
17992,86,16,E6P.B,C,C,0.0,0.0,0.0,-0.000000,-0.500000,-0.0,-0.000000,-0.166667,B,0.689852,0.43330,1.0
17993,87,13,E6P.B,C,C,0.0,0.0,0.0,-0.000000,-0.600000,-0.0,-0.000000,-0.000000,B,0.742278,0.43330,1.0


In [11]:
label="PDBID"
classes= df[label].unique().tolist()
print(f"Label classes: {classes}")
df[label]= df[label].map(classes.index)

Label classes: ['DZI.A', 'JTG.A', 'JTG.B', 'CDL.A', 'WPT.B', 'X4M.E', 'EAW.A', 'BGS.E', 'HHB.D', 'N8Z.B', 'JAT.B', 'YFD.B', 'DDM.B', 'AXI.A', 'HMY.C', 'BGS.A', 'U6H.E', 'NM8.B', 'F47.B', 'BRS.D', 'EBP.C', 'NPS.A', 'ES7.A', 'KSO.A', 'FZA.B', 'A22.A', 'Q3J.B', 'A4Y.B', 'NZ9.A', 'BP8.A', 'KRM.B', 'NGB.C', 'A4Y.A', 'DVF.B', 'FCC.C', 'JAT.A', 'K8R.A', 'KTZ.A', 'KTZ.B', 'MLC.B', 'REW.A', 'REW.B', 'REW.C', 'VFB.A', 'VFB.C', 'G2U.B', 'PCC.A', 'X0B.B', 'BN9.B', 'HHR.A', 'K2M.D', 'MBW.A', 'SAK.A', 'W2D.A', 'IOF.A', 'J2L.C', 'JEU.A', 'L72.A', 'LRX.A', 'O27.B', 'OZG.F', 'P23.C', 'PWX.C', 'UWQ.A', 'Y61.A', 'YEB.A', 'C6T.A', 'CYK.A', 'E6P.A', 'E6P.B']


In [12]:
X = df.drop(["Label", "Residue_Index","PDBID","Chain","Secondary_Structure","Junction_Region"], axis=1)
y = df["Label"]

In [13]:
X

,Amino_Acid,Arginine_Propensity,Tryptophan_Propensity,Tyrosine_Propensity,Valine_Propensity,Serine_Propensity,Methionine_Propensity,Threonine_Propensity,Leucine_Propensity,RelSASA,Secondary_Enum
0,1,0.0,0.0,0.0,-0.000000,-0.000000,-0.0,-0.000000,-0.000000,0.738996,0.43330
1,10,0.0,0.0,0.0,-0.000000,-0.000000,-0.0,-0.000000,-0.000000,0.417377,0.43330
2,8,0.0,0.0,0.0,-0.000000,-0.000000,-0.0,-0.000000,-0.000000,0.110747,6.27530
3,3,0.0,0.0,0.0,-0.428571,-0.000000,-0.0,-0.000000,-0.142857,0.094342,6.27530
4,18,0.0,0.0,0.0,-0.571429,-0.000000,-0.0,-0.000000,-0.142857,0.016116,0.17134
...,...,...,...,...,...,...,...,...,...,...,...
17990,6,0.0,0.0,0.0,-0.000000,-0.285714,-0.0,-0.142857,-0.285714,0.424894,0.43330
17991,16,0.0,0.0,0.0,-0.000000,-0.428571,-0.0,-0.000000,-0.285714,0.603341,0.43330
17992,16,0.0,0.0,0.0,-0.000000,-0.500000,-0.0,-0.000000,-0.166667,0.689852,0.43330
17993,13,0.0,0.0,0.0,-0.000000,-0.600000,-0.0,-0.000000,-0.000000,0.742278,0.43330


In [14]:
# import XGBoost
import xgboost as xgb

# # define data_dmatrix
data_dmatrix = xgb.DMatrix(data=X,label=y)

In [15]:
# split X and y into training and testing sets

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [16]:
# Calculate the ratio of negative class to positive class
ratio = float(y_train.value_counts()[0]) / y_train.value_counts()[1]
print(ratio)

8.727027027027027


In [40]:
# import XGBClassifier
from xgboost import XGBClassifier


# # declare parameters
# params = {
#             'objective':'binary:logistic',
#             'max_depth': 4,
#             'alpha': 10,
#             'learning_rate': 1.0,
#             'n_estimators':100
#         }



# instantiate the classifier
xgb_clf = XGBClassifier(learning_rate =0.1,
 n_estimators=100,
 max_depth=45,
 min_child_weight=1,
 gamma=0,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=13)



# fit the classifier to the training data
xgb_clf.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=45, max_leaves=None,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None, nthread=4,
              num_parallel_tree=None, ...)

In [41]:
# WITHOUT SCALE POS WEIGHT AND SMOTE
y_pred = xgb_clf.predict(X_test)

In [42]:
# check accuracy score
from sklearn.metrics import accuracy_score

print('XGBoost model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

XGBoost model accuracy score: 0.8955
              precision    recall  f1-score   support

         0.0       0.91      0.98      0.94      3229
         1.0       0.47      0.13      0.20       370

    accuracy                           0.90      3599
   macro avg       0.69      0.56      0.57      3599
weighted avg       0.86      0.90      0.87      3599



In [43]:
#WITH SCALE POS WEIGHT
model = xgb.XGBClassifier(learning_rate =0.1,
 n_estimators=100,
 max_depth=45,
 min_child_weight=1,
 gamma=0,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=ratio,
 seed=13)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [44]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
print('XGBoost model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))
print(classification_report(y_test, y_pred))

XGBoost model accuracy score: 0.8661
              precision    recall  f1-score   support

         0.0       0.91      0.94      0.93      3229
         1.0       0.28      0.19      0.23       370

    accuracy                           0.87      3599
   macro avg       0.60      0.57      0.58      3599
weighted avg       0.85      0.87      0.85      3599



In [33]:
df1 = pd.read_excel("viral-ml.xlsx")
df1

FileNotFoundError: [Errno 2] No such file or directory: 'viral-ml.xlsx'

In [ ]:
df1=df1.drop(["Predicted_Labels","SUM","Residue_Index", "PDBID"], axis=1)

In [ ]:
df1['Amino_Acid'] = df1['Amino_Acid'].map(aa_to_number)

In [ ]:
# label="PDBID"
# classes= df1[label].unique().tolist()
# print(f"Label classes: {classes}")
# df1[label]= df1[label].map(classes.index)


In [ ]:
df["Valine_Propensity"]=df["Valine_Propensity"].apply(lambda x:x*-1)
df["Serine_Propensity"]=df["Serine_Propensity"].apply(lambda x:x*-1)
df["Methionine_Propensity"]=df["Methionine_Propensity"].apply(lambda x:x*-1)
df["Threonine_Propensity"]=df["Threonine_Propensity"].apply(lambda x:x*-1)
df["Leucine_Propensity"]=df["Leucine_Propensity"].apply(lambda x:x*-1)
df

,Residue_Index,PDBID,Rel-SASA,Amino_Acid,Arginine_Propensity,Tryptophan_Propensity,Tyrosine_Propensity,Valine_Propensity,Serine_Propensity,Methionine_Propensity,Threonine_Propensity,Leucine_Propensity,Secondary_Enum,SUM,Label
0,1,0,0.922639,16,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.46007,0.460070,0.0
1,2,0,0.360844,10,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.46007,0.460070,0.0
2,3,0,0.669703,3,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.46007,0.460070,0.0
3,4,0,0.216953,8,0.0,0.0,0.0,0.0,0.285714,0.0,0.000000,0.285714,0.46007,-0.111359,0.0
4,5,0,0.527341,14,0.0,0.0,0.0,0.0,0.142857,0.0,0.000000,0.285714,0.46007,0.031499,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9485,46,58,0.419315,7,0.0,0.0,0.0,0.0,0.142857,0.0,0.142857,0.142857,0.46007,0.031499,0.0
9486,47,58,0.527842,5,0.0,0.0,0.0,0.0,0.142857,0.0,0.000000,0.142857,0.46007,0.174356,0.0
9487,48,58,0.360336,6,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.46007,0.460070,0.0
9488,49,58,0.744236,9,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.46007,0.460070,0.0


In [ ]:
predictions1 = model.predict(df1)

ValueError: feature_names mismatch: ['Residue_Index', 'PDBID', 'Rel-SASA', 'Amino_Acid', 'Arginine_Propensity', 'Tryptophan_Propensity', 'Tyrosine_Propensity', 'Valine_Propensity', 'Serine_Propensity', 'Methionine_Propensity', 'Threonine_Propensity', 'Leucine_Propensity', 'Secondary_Enum'] ['Rel-SASA', 'Amino_Acid', 'Arginine_Propensity', 'Tryptophan_Propensity', 'Tyrosine_Propensity', 'Valine_Propensity', 'Serine_Propensity', 'Methionine_Propensity', 'Threonine_Propensity', 'Leucine_Propensity', 'Secondary_Enum']
expected Residue_Index, PDBID in input data

In [ ]:
csv_file_path = "preds_scaleposw.csv"

# Create a DataFrame with the predicted labels
result_df = pd.DataFrame({'Predicted_Labels': predictions1.flatten()})

# Save the DataFrame to a CSV file
result_df.to_csv(csv_file_path, index=False)

# Print a message indicating the successful save
print(f"Predictions saved to {csv_file_path}")

In [ ]:
df2 = pd.read_excel("viral-ml-with-preds.xlsx")
df2['LUBNA'] = df2['LUBNA'].replace(np.nan, 0)

In [ ]:
test=df2['LUBNA']

In [ ]:
print(classification_report(test, predictions1))

In [ ]:
# bst = model.get_booster()
# bst.dump_model('dump.raw.txt')

In [ ]:
# model.get_booster().trees_to_dataframe()